In [4]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
if os.path.exists(".env"):
    load_dotenv()
    print("Environment variables loaded from .env file.")


# Access the API key
api_key = os.getenv("OPENAI_API_KEY")

if api_key:
    print("API Key loaded successfully!")
else:
    print("API Key not found. Please check your .env file.")

Environment variables loaded from .env file.
API Key loaded successfully!


In [7]:
import re
import io
import requests
from langchain_core.tools import tool
from typing import Annotated, Sequence, TypedDict, Optional

@tool
def read_document(
    file_name: Annotated[str, "File path to read the document from."],
    start: Annotated[Optional[int], "The start line. Default is 0"] = None,
    end: Annotated[Optional[int], "The end line. Default is None"] = None,
) -> str:
    """Read the specified document."""
    with (WORKING_DIRECTORY / file_name).open("r") as file:
        lines = file.readlines()
    if start is None:
        start = 0
    return "\n".join(lines[start:end])

from typing import Annotated, Optional
import shutil

from pathlib import Path

UPLOAD_DIRECTORY = Path("./Project_Agentic/") / "uploads"
UPLOAD_DIRECTORY.mkdir(parents=True, exist_ok=True)

@tool
def upload_document(
    source_path: Annotated[str, "Local file path to upload."],
    destination_name: Annotated[Optional[str], "Name to save the file as."] = None,
) -> str:
    """Upload a document into the working directory."""
    destination_name = destination_name or source_path.split("/")[-1]
    destination_path = UPLOAD_DIRECTORY / destination_name
    shutil.copy(source_path, destination_path)
    return f"File uploaded successfully as {destination_name}"







## Code Explaination + loop

       ┌──────────────────────┐
       │   User uploads file  │
       │  (upload_document)   │
       └─────────┬────────────┘
                 │
                 ▼
       ┌──────────────────────┐
       │  User sends query    │
       │ (answer_from_document)│
       └─────────┬────────────┘
                 │
                 ▼
       ┌──────────────────────┐
       │  LLM Node             │
       │ call_llm_node()       │
       │ - Receives messages   │
       │ - Generates response  │
       │ - May include tool_calls │
       └─────────┬────────────┘
                 │
                 ▼
       ┌──────────────────────┐
       │ Should Continue?      │
       │ should_continue()     │
       └───────┬───────┬──────┘
               │       │
          "continue"  "end"
               │       │
               ▼       ▼
       ┌──────────────┐   ┌─────────────────┐
       │ Tool Node    │   │ Final Answer    │
       │ tool_node()  │   │ (return to user)│
       │ Executes     │   └─────────────────┘
       │ tool calls   │
       └───────┬──────┘
               │
               ▼
       ┌──────────────────────┐
       │  Tool Outputs         │
       │ ToolMessage objects   │
       │ Added to messages     │
       └─────────┬────────────┘
                 │
                 ▼
       ┌──────────────────────┐
       │  Back to LLM Node    │
       │ call_llm_node()      │
       │ - Processes new      │
       │   messages including │
       │   tool results       │
       └─────────┬────────────┘
                 │
                 ▼
          (Loop until should_continue() returns "end")
